# 簡單的RNN實作

## 程式參考來源：
- https://keras.io/api/layers/core_layers/embedding/
- https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
- https://keras.io/guides/working_with_rnns/


## 載入相關套件

In [1]:
# 載入相關套件
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## 嵌入層測試

In [2]:
# 建立模型
model = tf.keras.Sequential()

# 模型只含嵌入層(Embedding layer)
# 字彙表最大為1000，輸出維度為 64，輸入的字數為 10
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# 產生亂數資料，32筆資料，每筆 10 個數字
input_array = np.random.randint(1000, size=(32, 10))

# 指定優化器、損失函數
model.compile('rmsprop', 'mse')

# 預測
output_array = model.predict(input_array)
print(output_array.shape)
output_array[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
(32, 10, 64)


array([[-3.74860875e-02, -4.93437164e-02, -1.94707755e-02,
         1.92413218e-02,  1.79875828e-02, -1.29656792e-02,
         1.56907551e-02,  2.98589803e-02, -2.49721296e-02,
        -3.09133772e-02,  2.90367343e-02,  3.66880931e-02,
         3.63899209e-02, -1.53474100e-02, -8.86081532e-03,
        -2.79420614e-03,  2.35770680e-02,  3.29114310e-02,
         4.23764847e-02, -2.66217478e-02, -7.52661377e-03,
        -4.75716591e-02,  9.54782963e-03, -1.31713860e-02,
         1.73690431e-02,  6.56251982e-03, -1.37666948e-02,
         3.40363272e-02,  1.36009790e-02,  2.19789855e-02,
        -3.49311456e-02, -1.69756263e-03,  4.29044403e-02,
         4.88670506e-02,  1.91758014e-02,  4.39615510e-02,
         3.44832428e-02, -2.96313409e-02, -3.98716927e-02,
         4.46766503e-02,  2.77245976e-02, -1.06930248e-02,
         4.63406108e-02, -8.00140947e-03,  3.22640873e-02,
         3.62134241e-02, -2.75740027e-02, -1.47160999e-02,
         2.57541575e-02,  2.35200562e-02,  4.74881865e-0

## 使用真實的資料轉換

In [11]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 測試資料
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better.']

# 轉成 one-hot encoding
vocab_size = 50 # 字典最大字數
encoded_docs = [one_hot(d, vocab_size) for d in docs]

# 轉成固定長度，長度不足則後面補空白
padded_docs = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')

# 模型只有 Embedding
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, 64))
model.compile('rmsprop', 'mse')

# 預測
output_array = model.predict(padded_docs)
output_array.shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


(10, 4, 64)

In [12]:
# one-hot encoding 轉換結果
print(encoded_docs[0])

# 補空白後的輸入維度
print(padded_docs.shape)

[13, 24]
(10, 4)


## 加上完全連接層(Dense)

In [13]:
# 定義 10 個語句的正面(1)或負面(0)的情緒
labels = np.array([1,1,1,1,1,0,0,0,0,0])

vocab_size = 50
maxlen = 4
encoded_docs = [one_hot(d, vocab_size) for d in docs]
padded_docs = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')

model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, 8))
model.add(layers.Flatten())

# 加上完全連接層(Dense)
model.add(layers.Dense(1, activation='sigmoid'))

# 指定優化器、損失函數
model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy'])

print(model.summary())

# 模型訓練
model.fit(padded_docs, labels, epochs=50, verbose=0)

# 模型評估
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print(f'Accuracy: {accuracy*100}%')

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Accuracy: 100.0%


In [6]:
model.predict(padded_docs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


array([[0.54022646],
       [0.5319353 ],
       [0.51778364],
       [0.53849167],
       [0.5297085 ],
       [0.49326497],
       [0.47794208],
       [0.46133637],
       [0.4992804 ],
       [0.41422594]], dtype=float32)

## 加上 RNN 神經層

In [7]:
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, 8, input_length=maxlen))

# 加上 RNN 神經層，輸出 128 個神經元
model.add(layers.SimpleRNN(128))

# 加上完全連接層(Dense)
model.add(layers.Dense(1, activation='sigmoid'))

# 指定優化器、損失函數
model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy'])

print(model.summary())
# 模型訓練
model.fit(padded_docs, labels, epochs=50, verbose=0)

# 模型評估
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Accuracy: 100.000000


In [8]:
model.predict(padded_docs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


array([[9.9998039e-01],
       [9.9996644e-01],
       [9.9997205e-01],
       [9.9995279e-01],
       [9.9997377e-01],
       [8.2181134e-05],
       [2.6281900e-06],
       [2.6986641e-05],
       [4.8819020e-06],
       [2.2153838e-05]], dtype=float32)

## 使用詞向量(Word2Vec)

## 讀取 GloVe 300維的詞向量，產生字典資料型變數，方便搜尋

In [15]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('../GloVe/glove.6B.300d.txt', encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.array(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

## 分詞

In [16]:
# 分詞
from tensorflow.keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(docs)

vocab_size = len(t.word_index) + 1

# 轉為序列整數
encoded_docs = t.texts_to_sequences(docs)

# 補 0
padded_docs = pad_sequences(encoded_docs, maxlen=maxlen, padding='post')
padded_docs

array([[ 6,  2,  0,  0],
       [ 3,  1,  0,  0],
       [ 7,  4,  0,  0],
       [ 8,  1,  0,  0],
       [ 9,  0,  0,  0],
       [10,  0,  0,  0],
       [ 5,  4,  0,  0],
       [11,  3,  0,  0],
       [ 5,  1,  0,  0],
       [12, 13,  2, 14]])

## 轉換為GloVe 300維的詞向量

In [17]:
# 轉換為 GloVe 300維的詞向量
# 初始化輸出
embedding_matrix = np.zeros((vocab_size, 300))

# 讀取詞向量值
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# 任取一筆觀察        
embedding_matrix[2]

array([ 0.19205999,  0.16459   ,  0.060122  ,  0.17696001, -0.27405   ,
        0.079646  , -0.25292999, -0.11763   ,  0.17614   , -1.97870004,
        0.10707   , -0.028088  ,  0.093991  ,  0.48135   , -0.037581  ,
        0.0059231 , -0.11118   , -0.099847  , -0.22189   ,  0.0062044 ,
        0.17721   ,  0.25786   ,  0.42120999, -0.13085   , -0.32839   ,
        0.39208999, -0.050214  , -0.46766999, -0.063107  , -0.0023065 ,
        0.21005   ,  0.26982   , -0.22652   , -0.42958999, -0.89682001,
        0.21932   , -0.0020377 ,  0.1358    , -0.12661999, -0.058927  ,
        0.0049502 , -0.28457999, -0.29530999, -0.29295999, -0.24212   ,
        0.091915  ,  0.01977   ,  0.14503001,  0.26495999,  0.10817   ,
        0.029115  ,  0.075254  ,  0.16463999,  0.12097   , -0.37494001,
        0.52671999,  0.094318  , -0.054813  , -0.021008  ,  0.081353  ,
        0.18735   , -0.14458001, -0.031203  ,  0.31753999,  0.027703  ,
       -0.28657001,  0.34630999, -0.27772   ,  0.18669   , -0.11

## Embedding 設為不需訓練，直接輸入轉換後的向量

In [18]:
model = tf.keras.Sequential()
# trainable=False：不需訓練，直接輸入轉換後的向量
model.add(layers.Embedding(vocab_size, 300, weights=[embedding_matrix], 
                           trainable=False))
model.add(layers.SimpleRNN(128))
model.add(layers.Dense(1, activation='sigmoid'))

# 指定優化器、損失函數
model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy'])

print(model.summary())

# 模型訓練
model.fit(padded_docs, labels, epochs=50, verbose=0)

# 模型評估
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print(f'Accuracy: {accuracy*100}%')

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ ?                           │           4,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,500 (17.58 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,500 (17.58 KB)

None
Accuracy: 100.0%


In [20]:
model.predict(padded_docs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


array([[9.99894798e-01],
       [9.99870718e-01],
       [9.99929249e-01],
       [9.99948323e-01],
       [9.99944687e-01],
       [6.61946542e-05],
       [5.28137425e-05],
       [1.69350678e-04],
       [1.02341211e-04],
       [1.11033136e-04]], dtype=float32)

## 雙向(Bidirectional)

In [18]:
model = tf.keras.Sequential()
# trainable=False：不需訓練，直接輸入轉換後的向量
model.add(layers.Embedding(vocab_size, 300, weights=[embedding_matrix], 
                           trainable=False))
model.add(layers.Bidirectional(layers.SimpleRNN(128)))
model.add(layers.Dense(1, activation='sigmoid'))

# 指定優化器、損失函數
model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy'])

print(model.summary())

# 模型訓練
model.fit(padded_docs, labels, epochs=50, verbose=0)

# 模型評估
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print(f'Accuracy: {accuracy*100}%')

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ ?                           │           4,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,500 (17.58 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,500 (17.58 KB)

None
Accuracy: 100.0%
